In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from plex_pipe.utils.config_loaders import load_analysis_settings
from plex_pipe.utils.globus_utils import GlobusConfig, create_globus_tc
from plex_pipe.utils.file_utils import globus_dir_exists, GlobusPathConverter
from plex_pipe.core_cutting.file_io import GlobusFileStrategy

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [2]:
# experiment settings file path
settings_path = r'C:\BLCA\BLCA-1A_Analysis\analysis_BLCA1A.yaml'

# globus config
config_path = Path(r'D:\globus_config\globus_config.yaml')

In [3]:
# load analysis configuration
settings = load_analysis_settings(settings_path)
settings

2026-01-06 10:54:12.190 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:51 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/BLCA-1A/BLCA-1A_Final', analysis_name='BLCA-1A_Analysis', local_analysis_dir='C:/BLCA', remote_analysis_dir='/ix1/kkedziora/BLCA', log_dir=None), core_detection=CoreDetectionSettings(detection_image='BLCA-1A_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=20000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=['008_ECad'], use_markers=None, ignore_markers=['Antibody1', 'TNFa', 'Snail1', 'SKP2', 'ProgRc', 'Plk1', 'PH3', 'PDL1', 'p65', 'p130', 'p-p130', 'p-Cdc6', 'LAG3', 'IL-8', 'HER2', 'ERa', 'EpCAM', 'E2F1', 'cycD3', 'cycB2', 'CDC25C', 'CD86', 'CD73', 'CD69', 'CD62L', 'CD56', 'CD4', 'CD25', 'CD19', 'CD27', 'CCR7', 'cCASP3'], margin=0, mask_value=0, transfer_cleanup_enabled=True, temp_roi_delete=True), additional_elements=[NormalizeStep(category='image_transformer'

In [4]:
# get Globus settings
gc = GlobusConfig.from_config_files(config_path, from_collection = 'cbi_collection_id', to_collection = 'crcd_collection_id')
tc = create_globus_tc(gc.client_id, gc.transfer_tokens)

In [6]:
# check if the destination analysis directory exists 
remote_analysis_dir = settings.general.remote_analysis_dir + '/' + settings.general.analysis_name

globus_dir_exists(tc, gc.destination_collection_id, remote_analysis_dir)

True

In [7]:
# build a transfer map

# if transfering from Windows with multiple drives exposed in Globus use "multi_drive"
# if transfering from Windows with a single drive exposed in Globus 
# or if transfering from CIFs drive use "single_drive"
conv = GlobusPathConverter(layout="multi_drive")

local_settings_path = conv.windows_to_globus(settings_path)
remote_settings_path = remote_analysis_dir + '/' + Path(settings_path).name
    
local_cores_info_path = conv.windows_to_globus(settings.core_info_file_path.with_suffix('.pkl'))
remote_cores_info_path = remote_analysis_dir + '/' + Path(local_cores_info_path).name

remote_cores_info_path = remote_analysis_dir + '/' + Path(local_cores_info_path).name

transfer_map = {}
transfer_map['settings'] = (local_settings_path, remote_settings_path)
transfer_map['core_definitions'] = (local_cores_info_path, remote_cores_info_path)
transfer_map

{'settings': ('/C/BLCA/BLCA-1A_Analysis/analysis_BLCA1A.yaml',
  '/ix1/kkedziora/BLCA/BLCA-1A_Analysis/analysis_BLCA1A.yaml'),
 'core_definitions': ('/C/BLCA/BLCA-1A_Analysis/cores.pkl',
  '/ix1/kkedziora/BLCA/BLCA-1A_Analysis/cores.pkl')}

In [8]:
# submit transfer
transfer = GlobusFileStrategy(tc=tc, transfer_map=transfer_map, gc=gc, cleanup_enabled = False)

2026-01-06 10:58:22.536 | INFO     | plex_pipe.core_cutting.file_io:submit_all_transfers:91 - Submitted transfer for settings to /ix1/kkedziora/BLCA/BLCA-1A_Analysis/analysis_BLCA1A.yaml (task_id=8641ef5f-eb18-11f0-b1af-0213754b0ca1)
2026-01-06 10:58:23.031 | INFO     | plex_pipe.core_cutting.file_io:submit_all_transfers:91 - Submitted transfer for core_definitions to /ix1/kkedziora/BLCA/BLCA-1A_Analysis/cores.pkl (task_id=86925b1f-eb18-11f0-bdb2-0213754b0ca1)


In [11]:
# monitor transfer
print(f'Status of {transfer.pending[0][1]}: {tc.get_task(transfer.pending[0][0])["status"]}')
print(f'Status of {transfer.pending[1][1]}: {tc.get_task(transfer.pending[1][0])["status"]}')

Status of /ix1/kkedziora/BLCA/BLCA-1A_Analysis/analysis_BLCA1A.yaml: SUCCEEDED
Status of /ix1/kkedziora/BLCA/BLCA-1A_Analysis/cores.pkl: SUCCEEDED
